In [151]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import XSD #most common namespaces
import pandas as pd
#import rdflib
#from rdflib.namespace import NamespaceManager

In [152]:
cg = CoinGeckoAPI()

In [153]:
coins = cg.get_coins_list(include_platform=False)
coin_list = [c['id'] for c in coins]

In [154]:
whitepapers = ['aave','adex','aditus','bitcoin','ethereum','0x','aichain','ark','polkadot']

In [155]:
coin_list = [x for x in coin_list if x in whitepapers]

In [156]:
coins = pd.DataFrame()
for coin in coin_list:
    print(coin)
    coin_info = cg.get_coin_by_id(id=coin.lower(), localization=False, tickers=False, market_data=True,
                                  community_data=True, developer_data=True, sparkline=False)
    df = pd.DataFrame.from_dict(coin_info, orient='index')
    df = df.T
    coins = coins.append(df)

0x
aave
adex
aditus
aichain
ark
bitcoin
ethereum
polkadot


In [157]:
for column in coins.columns:
    try:
        coins[column] = coins[column].str.replace(' ', '')
        coins[column] = coins[column].str.replace('\\', '')
        coins[column] = coins[column].str.replace('-', '')
        coins[column] = coins[column].str.lower()
    except:
        pass

<ipython-input-157-aad7fb239d62>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  coins[column] = coins[column].str.replace('\\', '')


In [158]:
g = Graph()
schema = Namespace('http://schema.org/')

In [159]:
for index, row in coins.iterrows():
        #Make the symbol column a class
        g.add((URIRef(row['symbol']), RDF.type, URIRef(schema + 'symbol')))
        #Connect symbol to the id column and define id as a string
        g.add((URIRef(row['symbol']), URIRef(schema+'id'), Literal(row['id'], datatype=XSD.string) ))
        
        #For coins that have a value for 'asset_platform_id', define asset_platform_id as a class and connect to the symbol
        if row['asset_platform_id'] != None:
            g.add((URIRef(row['asset_platform_id']), RDF.type, URIRef(schema+'asset_platform_id')))
            g.add((URIRef(row['symbol']), URIRef(schema+'asset_platform_id'), Literal(row['asset_platform_id'], datatype=XSD.string) ))  
        
        #For coins that have a value for 'coingecko_rank', connect this value to symbol as a literal
        if row['coingecko_rank'] != None:
            try:
                g.add((URIRef(row['symbol']), URIRef(schema+'coingecko_rank'), Literal(row['coingecko_rank'], datatype=XSD.integer) ))
            except:
                pass
        #For coins that have a value for 'liquidity_score', connect this value to symbol as a literal
        if row['liquidity_score'] != None:
            try:
                g.add((URIRef(row['symbol']), URIRef(schema+'liquidity_score'), Literal(row['liquidity_score'], datatype=XSD.float) ))
            except:
                pass            

In [160]:
#print(g.serialize(format='turtle'))

In [161]:
df = pd.read_csv("allMentions.csv")

In [162]:
df.head()

,Unnamed: 0,mentions,token
0,0,BitHope,adx
1,1,CryptoCrowd,adx
2,2,RSK,adx
3,3,po,adx
4,0,httpswww,ark


In [163]:
for column in  df.columns:
    try:
        df[column] = df[column].str.replace(' ', '')
        df[column] = df[column].str.replace('\\', '')
        df[column] = df[column].str.replace('-', '')
        df[column] = df[column].str.lower()
    except:
        pass

<ipython-input-163-6314741b7189>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column] = df[column].str.replace('\\', '')


In [164]:
df['mentions'] = df['mentions'].str.replace('bitcoin', 'btc')
df['token'] = df['token'].str.replace('bitcoin', 'btc')


In [165]:
df

,Unnamed: 0,mentions,token
0,0,bithope,adx
1,1,cryptocrowd,adx
2,2,rsk,adx
3,3,po,adx
4,0,httpswww,ark
...,...,...,...
893,217,gamematrix,ait
894,218,governmentbusiness,ait
895,219,hain,ait
896,220,huoyan,ait


In [166]:
for index, row in df.iterrows():
    
        #Make symbol a class - the same as symbol from the metadata so that they are connected in the RDF
        g.add((URIRef(row['token']), RDF.type, URIRef(schema + 'symbol')))
        #Add connection from token to mentions
        g.add((URIRef(row['token']), URIRef(schema+'mentions'), URIRef(schema + row['mentions']) ))
        #g.add((URIRef(row['token']), URIRef(schema+'mentions'), Literal(row['mentions'], datatype=XSD.string) ))


In [167]:
g.serialize('KG.ttl',format='turtle')

<Graph identifier=N4b3bc41124d34b83bcfd20a8181b6138 (<class 'rdflib.graph.Graph'>)>